# Car price prediction project (module 5)

## Import

In [135]:
# general packages
import time
import re

import numpy as np
from IPython.display import display

# tabular data analysis/processing
import pandas as pd
from pandas_profiling import ProfileReport

# visualization

print('Import is completed.')

Import is completed.


## Functions

In [136]:
def exists(var_name:str):
    """Check if a variable with var_name exists in global scope"""
    return var_name in globals()

def print_framed(text: str, frame_width=100):
    text_width = len(text)
    gap = 5
    edge_width = (frame_width - text_width) // 2 - gap
    print('*'*frame_width)
    print('*'*edge_width + ' '*gap + text + ' '*gap + '*'*edge_width)
    print('*'*frame_width)



## Init setup

In [137]:
# fix random seed
SEED = 37

# # number of threads for computations
# N_THREADS = max(1, multiprocessing.cpu_count() - 1)

# fix requirements
!pip freeze > requirements.txt

## Data loading

Load parsed in [separate notebook](https://github.com/PavelKucherenko/sf-ds50-course/blob/main/module_5/Parsing/Parsing.ipynb) train data and provided test project data.

In [138]:
TRAIN_DIR = 'Parsing/Data'
TEST_DIR = 'Data'

In [139]:
# del df_test_orig, df_train_orig
print_framed('TRAIN DATASET')
if not exists('df_train_orig'): df_train_orig = pd.read_csv(f'{TRAIN_DIR}/parsed_car_data__17_55__02_02_2022.csv')
# noinspection PyUnboundLocalVariable
display(df_train_orig.head(5))
df_train_orig.info()
display(df_train_orig.describe())

print_framed('TEST DATASET')
if not exists('df_test_orig'): df_test_orig = pd.read_csv(f'{TEST_DIR}/test.csv')
# noinspection PyUnboundLocalVariable
display(df_test_orig.head(5))
df_test_orig.info()
display(df_test_orig.describe())

****************************************************************************************************
**************************************     TRAIN DATASET     **************************************
****************************************************************************************************


,№,bodyType,brand,car_url,color,complectation_dict,description,engineDisplacement,enginePower,equipment_dict,...,vehicleTransmission,vendor,Владельцы,Владение,ПТС,Привод,Руль,Состояние,Таможня,Price
0,0,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,коричневый,"{'id': '20291328', 'name': 'Laurin&Klement', '...",1 владелец.\r\nРодной пробег.\r\nПТС оригинал....,1.8,180.0,"{'cruise-control': True, 'esp': True, 'adaptiv...",...,роботизированная,NaN,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен,1030000.0
1,1,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,красный,{'id': '0'},Skoda Octavia\r\n\r\n❕ ПТС ОРИГИНАЛ\r\n❕ ДВА В...,1.8,150.0,"{'condition': True, 'mirrors-heat': True, 'ele...",...,механическая,NaN,2 владельца,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен,400000.0
2,2,NaN,SKODA,https://auto.ru/cars/new/group/skoda/karoq/225...,белый,"{'id': '22687243', 'name': 'Hockey Edition', '...",В стоимость автомобиля включено дополнительное...,NaN,NaN,"{'cruise-control': True, 'asr': True, 'tinted-...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2655000.0
3,3,NaN,SKODA,https://auto.ru/cars/new/group/skoda/rapid/217...,коричневый,"{'id': '22870340', 'name': 'Hockey Edition', '...",———————————————————————————\r\nФУНКЦИОНАЛЬНОЕ ...,NaN,NaN,"{'cruise-control': True, 'asr': True, 'tinted-...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1335000.0
4,4,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/rapid/110...,чёрный,{'id': '0'},ЛОТ: 01306544\r\nЛокация №16\r\nФаворит Моторс...,1.6,110.0,"{'cruise-control': True, 'glonass': True, 'asr...",...,автоматическая,NaN,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен,1820000.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42305 entries, 0 to 42304
Data columns (total 34 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   №                     42305 non-null  int64  
 1   bodyType              33195 non-null  object 
 2   brand                 41635 non-null  object 
 3   car_url               42305 non-null  object 
 4   color                 41635 non-null  object 
 5   complectation_dict    38510 non-null  object 
 6   description           41635 non-null  object 
 7   engineDisplacement    32624 non-null  float64
 8   enginePower           32624 non-null  float64
 9   equipment_dict        41552 non-null  object 
 10  fuelType              32624 non-null  object 
 11  image                 30612 non-null  object 
 12  mileage               33195 non-null  float64
 13  modelDate             40061 non-null  float64
 14  model_info            41552 non-null  object 
 15  model_name         

,№,engineDisplacement,enginePower,mileage,modelDate,numberOfDoors,parsing_unixtime,productionDate,sell_id,vehicleConfiguration,vendor,Price
count,42305.000000,32624.000000,32624.000000,3.319500e+04,40061.000000,41552.000000,4.230500e+04,41635.000000,0.0,0.0,0.0,3.765800e+04
mean,21152.000000,2.382127,193.439554,1.310194e+05,2012.156411,4.650823,1.643819e+09,2014.446019,NaN,NaN,NaN,2.780145e+06
std,12212.545906,0.970412,87.901146,9.343720e+04,6.814103,0.579108,2.877574e+03,6.738529,NaN,NaN,NaN,2.896566e+06
min,0.000000,0.700000,45.000000,1.000000e+03,1937.000000,2.000000,1.643814e+09,1948.000000,NaN,NaN,NaN,4.000000e+04
25%,10576.000000,1.800000,136.000000,6.100000e+04,2009.000000,4.000000,1.643816e+09,2011.000000,NaN,NaN,NaN,1.090000e+06
50%,21152.000000,2.000000,170.000000,1.160000e+05,2014.000000,5.000000,1.643819e+09,2016.000000,NaN,NaN,NaN,1.930000e+06
75%,31728.000000,3.000000,240.000000,1.820000e+05,2017.000000,5.000000,1.643821e+09,2020.000000,NaN,NaN,NaN,3.450000e+06
max,42304.000000,6.600000,900.000000,1.090000e+06,2021.000000,5.000000,1.643824e+09,2022.000000,NaN,NaN,NaN,1.111000e+08


****************************************************************************************************
***************************************     TEST DATASET     ***************************************
****************************************************************************************************


,bodyType,brand,car_url,color,complectation_dict,description,engineDisplacement,enginePower,equipment_dict,fuelType,...,vehicleConfiguration,vehicleTransmission,vendor,Владельцы,Владение,ПТС,Привод,Руль,Состояние,Таможня
0,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,синий,NaN,"Все автомобили, представленные в продаже, прох...",1.2 LTR,105 N12,"{""engine-proof"":true,""tinted-glass"":true,""airb...",бензин,...,LIFTBACK ROBOT 1.2,роботизированная,EUROPEAN,3 или более,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен
1,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,чёрный,NaN,ЛОТ: 01217195\nАвтопрага Север\nДанный автомоб...,1.6 LTR,110 N12,"{""cruise-control"":true,""asr"":true,""esp"":true,""...",бензин,...,LIFTBACK MECHANICAL 1.6,механическая,EUROPEAN,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен
2,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/superb/11...,серый,"{""id"":""20026336"",""name"":""Ambition"",""available_...","Все автомобили, представленные в продаже, прох...",1.8 LTR,152 N12,"{""cruise-control"":true,""tinted-glass"":true,""es...",бензин,...,LIFTBACK ROBOT 1.8,роботизированная,EUROPEAN,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен
3,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,коричневый,"{""id"":""20803582"",""name"":""Ambition"",""available_...",КОМПЛЕКТ ЗИМНЕЙ (ЛЕТНЕЙ) РЕЗИНЫ ПО СЕЗОНУ В ПО...,1.6 LTR,110 N12,"{""cruise-control"":true,""roller-blind-for-rear-...",бензин,...,LIFTBACK AUTOMATIC 1.6,автоматическая,EUROPEAN,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен
4,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,белый,NaN,ЛОТ: 01220889\nАвтопрага Север\n\nВы можете по...,1.8 LTR,152 N12,"{""cruise-control"":true,""asr"":true,""esp"":true,""...",бензин,...,LIFTBACK AUTOMATIC 1.8,автоматическая,EUROPEAN,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34686 entries, 0 to 34685
Data columns (total 32 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   bodyType              34686 non-null  object
 1   brand                 34686 non-null  object
 2   car_url               34686 non-null  object
 3   color                 34686 non-null  object
 4   complectation_dict    6418 non-null   object
 5   description           34686 non-null  object
 6   engineDisplacement    34686 non-null  object
 7   enginePower           34686 non-null  object
 8   equipment_dict        24690 non-null  object
 9   fuelType              34686 non-null  object
 10  image                 34686 non-null  object
 11  mileage               34686 non-null  int64 
 12  modelDate             34686 non-null  int64 
 13  model_info            34686 non-null  object
 14  model_name            34686 non-null  object
 15  name                  34686 non-null

,mileage,modelDate,numberOfDoors,parsing_unixtime,productionDate,sell_id
count,34686.000000,34686.000000,34686.000000,3.468600e+04,34686.000000,3.468600e+04
mean,162009.767889,2007.074728,4.450816,1.603287e+09,2009.264602,1.098300e+09
std,100676.559489,7.415894,0.703040,1.493071e+05,7.047661,1.911225e+07
min,1.000000,1904.000000,0.000000,1.603107e+09,1904.000000,2.665000e+03
25%,91153.500000,2004.000000,4.000000,1.603221e+09,2006.000000,1.099049e+09
50%,149779.500000,2008.000000,5.000000,1.603254e+09,2011.000000,1.100911e+09
75%,215000.000000,2012.000000,5.000000,1.603290e+09,2014.000000,1.101245e+09
max,1000000.000000,2020.000000,5.000000,1.603710e+09,2020.000000,1.101375e+09


Thanks to parse stage we have all necessary columns in train set to predict car price based on test data. But we'll look more thoroughly later on.

## Preliminary data analysis and preprocessing

In [140]:
# create work copy of datasets
df_train = df_train_orig.copy()
df_test = df_test_orig.copy()

In [141]:
# and their combined list. Without merging them into one dataframe to avoid data leakage.
dfs = [df_train, df_test]

Now we have working copy of our dataset and a combined list.
NB! For all common actions to be done on BOTH datasets we'll use this list. /Was searching for best practice here and this is my choice for now/

### Data cleaning

There are some unuseful columns in both datasets. Let's just define them and delete.

In [142]:
# run profiling on both sets
# ProfileReport(df_train).to_file(output_file='Reports/train_profiling.html')
# ProfileReport(df_test).to_file(output_file='Reports/test_profiling.html')

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

So, analyzing pandas profiling result, there are following columns to DELETE from both datasets:
'car_url'               - potentially we could get some additional info from car page but for this case we don't need this "overhead"
'complectation_dict'    - 81.5% missing in test - too many
'equipment_dict'        - 28.8%  missing in test. Theoretically we could take from test set records without this missings and predict via specific model, which was trained with this
                        feature (there are only 1.8% missing values in train), but for this study case let's delete this column due to large amount of missing values anyway
'image'                 - don't go with CV for now, so don't need this column
'model_info'            - have potentially useful information from here in other columns
'name'                  - have this information in other columns
'priceCurrency'         - rubles everywhere (in test and in train)
'sell_id'               - not useful
'vehicleConfiguration'  - have this information in other columns
'Владение'              - 65.4% misssing in test
'Состояние'             - single-valued column in test and in train
'Таможня'               - single-valued column in test and in train

In [143]:
# as a result - cols to be deleted from both sets:
cols_to_delete = ['car_url',
                  'complectation_dict',
                  'equipment_dict',
                  'image',
                  'model_info',
                  'name',
                  'priceCurrency',
                  'sell_id',
                  'vehicleConfiguration',
                  'Владение',
                  'Состояние',
                  'Таможня']
for df in dfs:
    df.drop(columns=cols_to_delete, axis=1, inplace=True)
    df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42305 entries, 0 to 42304
Data columns (total 22 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   №                    42305 non-null  int64  
 1   bodyType             33195 non-null  object 
 2   brand                41635 non-null  object 
 3   color                41635 non-null  object 
 4   description          41635 non-null  object 
 5   engineDisplacement   32624 non-null  float64
 6   enginePower          32624 non-null  float64
 7   fuelType             32624 non-null  object 
 8   mileage              33195 non-null  float64
 9   modelDate            40061 non-null  float64
 10  model_name           41552 non-null  object 
 11  numberOfDoors        41552 non-null  float64
 12  parsing_unixtime     42305 non-null  int64  
 13  productionDate       41635 non-null  float64
 14  super_gen            41552 non-null  object 
 15  vehicleTransmission  33195 non-null 

Ok, it's easier now)

In [144]:
# also delete number column from train
df_train.drop(columns=['№'], axis=1, inplace=True)

In [145]:
# and check if the columns are equal and with same order (without last target 'price' in train)
list(df_train.columns[:-1]) == list(df_test.columns)

True

Ok, we have same features in train and in test.

Now delete missing price rows in train set.

In [152]:
df_train.shape

(42305, 21)

In [153]:
df_train[df_train['Price'].isna()].shape

(4647, 21)

In [154]:
df_train = df_train[~df_train['Price'].isna()]
df_train.shape

(37658, 21)

We also have missing bodyType in train set, let's deal with it.

In [158]:
df_train[df_train['bodyType'].isna()].shape

(7232, 21)

In [159]:
df_train[df_train['bodyType'].isna()]

,bodyType,brand,color,description,engineDisplacement,enginePower,fuelType,mileage,modelDate,model_name,...,parsing_unixtime,productionDate,super_gen,vehicleTransmission,vendor,Владельцы,ПТС,Привод,Руль,Price
2,NaN,SKODA,белый,В стоимость автомобиля включено дополнительное...,NaN,NaN,NaN,NaN,2017.0,KAROQ,...,1643813748,2022.0,"{'gear_type': 'FORWARD_CONTROL', 'engine_type'...",NaN,NaN,NaN,NaN,NaN,NaN,2655000.0
3,NaN,SKODA,коричневый,———————————————————————————\r\nФУНКЦИОНАЛЬНОЕ ...,NaN,NaN,NaN,NaN,2020.0,RAPID,...,1643813747,2022.0,"{'gear_type': 'FORWARD_CONTROL', 'engine_type'...",NaN,NaN,NaN,NaN,NaN,NaN,1335000.0
18,NaN,SKODA,синий,В стоимость автомобиля включено дополнительное...,NaN,NaN,NaN,NaN,2020.0,RAPID,...,1643813748,2022.0,"{'gear_type': 'FORWARD_CONTROL', 'engine_type'...",NaN,NaN,NaN,NaN,NaN,NaN,1815000.0
19,NaN,SKODA,коричневый,В стоимость автомобиля включено дополнительное...,NaN,NaN,NaN,NaN,2020.0,RAPID,...,1643813748,2022.0,"{'gear_type': 'FORWARD_CONTROL', 'engine_type'...",NaN,NaN,NaN,NaN,NaN,NaN,1535000.0
24,NaN,SKODA,серебристый,В стоимость автомобиля включено дополнительное...,NaN,NaN,NaN,NaN,2020.0,RAPID,...,1643813752,2022.0,"{'gear_type': 'FORWARD_CONTROL', 'engine_type'...",NaN,NaN,NaN,NaN,NaN,NaN,1995000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42284,NaN,MITSUBISHI,серый,Специальные условия на покупку Mitsubishi ASX ...,NaN,NaN,NaN,NaN,2020.0,ASX,...,1643823708,2021.0,"{'gear_type': 'ALL_WHEEL_DRIVE', 'engine_type'...",NaN,NaN,NaN,NaN,NaN,NaN,2105000.0
42288,NaN,MITSUBISHI,серый,Специальные условия на покупку Mitsubishi ASX ...,NaN,NaN,NaN,NaN,2020.0,ASX,...,1643823709,2021.0,"{'gear_type': 'FORWARD_CONTROL', 'engine_type'...",NaN,NaN,NaN,NaN,NaN,NaN,1670000.0
42292,NaN,MITSUBISHI,оранжевый,Специальные условия на покупку Mitsubishi ASX ...,NaN,NaN,NaN,NaN,2020.0,ASX,...,1643823708,2021.0,"{'gear_type': 'ALL_WHEEL_DRIVE', 'engine_type'...",NaN,NaN,NaN,NaN,NaN,NaN,1920000.0
42300,NaN,MITSUBISHI,белый,В честь 30-летия бренда Mitsubishi в России ст...,NaN,NaN,NaN,NaN,2020.0,ASX,...,1643823712,2021.0,"{'gear_type': 'ALL_WHEEL_DRIVE', 'engine_type'...",NaN,NaN,NaN,NaN,NaN,NaN,2246000.0


In [160]:
df_train[df_train['bodyType'].isna()]['productionDate'].unique()

array([2022., 2021., 2020., 2019.])

In [161]:
df_train[df_train['bodyType'].isna()]['Владельцы'].unique()

array([nan], dtype=object)

In [162]:
df_test['Владельцы'].unique()

array(['3 или более', '1\xa0владелец', '2\xa0владельца'], dtype=object)

So we can conclude that this records are for new cars, but we have no new cars in our test - delete this rows

In [163]:
df_train = df_train[~df_train['bodyType'].isna()]
df_train.shape

(30426, 21)

### Check and process duplicates

In [164]:
sum(df_train.duplicated())

39

In [165]:
# what are they..?
df_train[df_train.duplicated(keep=False) & ~df_train['bodyType'].isna()]

,bodyType,brand,color,description,engineDisplacement,enginePower,fuelType,mileage,modelDate,model_name,...,parsing_unixtime,productionDate,super_gen,vehicleTransmission,vendor,Владельцы,ПТС,Привод,Руль,Price
1021,лифтбек,SKODA,белый,...\r\nУникальное предложение на автомобили с ...,1.0,110.0,Бензин,39000.0,2017.0,RAPID,...,1643814001,2019.0,"{'gear_type': 'FORWARD_CONTROL', 'engine_type'...",механическая,NaN,2 владельца,Оригинал,передний,Левый,1290000.0
1036,лифтбек,SKODA,белый,...\r\nУникальное предложение на автомобили с ...,1.0,110.0,Бензин,39000.0,2017.0,RAPID,...,1643814001,2019.0,"{'gear_type': 'FORWARD_CONTROL', 'engine_type'...",механическая,NaN,2 владельца,Оригинал,передний,Левый,1290000.0
1240,лифтбек,SKODA,белый,"АВТОВЕК, являясь официальным дилером LADA, пре...",1.8,180.0,Бензин,61000.0,2017.0,OCTAVIA,...,1643814054,2018.0,"{'gear_type': 'FORWARD_CONTROL', 'engine_type'...",роботизированная,NaN,1 владелец,Оригинал,передний,Левый,1850000.0
1259,лифтбек,SKODA,белый,"АВТОВЕК, являясь официальным дилером LADA, пре...",1.8,180.0,Бензин,61000.0,2017.0,OCTAVIA,...,1643814054,2018.0,"{'gear_type': 'FORWARD_CONTROL', 'engine_type'...",роботизированная,NaN,1 владелец,Оригинал,передний,Левый,1850000.0
1962,лифтбек,SKODA,белый,✅ ПТС оригинал\r\n\r\n✅Хорошее техническое сос...,1.6,90.0,Бензин,64000.0,2017.0,RAPID,...,1643814235,2017.0,"{'gear_type': 'FORWARD_CONTROL', 'engine_type'...",механическая,NaN,3 или более,Оригинал,передний,Левый,920000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37214,седан,VOLKSWAGEN,белый,Volkswagen Passat 2012г.в. нaxодитcя по адреc...,1.4,122.0,Бензин,151000.0,2011.0,PASSAT,...,1643822550,2012.0,"{'gear_type': 'FORWARD_CONTROL', 'engine_type'...",роботизированная,NaN,3 или более,Оригинал,передний,Левый,790000.0
40571,внедорожник 5 дв.,MITSUBISHI,чёрный,Безопасность: 11. Обзор: 4. Комфорт: 15. Элеме...,2.4,181.0,Дизель,31000.0,2015.0,PAJERO_SPORT,...,1643823314,2018.0,"{'gear_type': 'ALL_WHEEL_DRIVE', 'engine_type'...",автоматическая,NaN,1 владелец,Оригинал,полный,Левый,2700000.0
40577,внедорожник 5 дв.,MITSUBISHI,чёрный,Безопасность: 11. Обзор: 4. Комфорт: 15. Элеме...,2.4,181.0,Дизель,31000.0,2015.0,PAJERO_SPORT,...,1643823314,2018.0,"{'gear_type': 'ALL_WHEEL_DRIVE', 'engine_type'...",автоматическая,NaN,1 владелец,Оригинал,полный,Левый,2700000.0
41621,внедорожник 5 дв.,MITSUBISHI,коричневый,Автомобиль в отличном состоянии. Бережная эксп...,1.6,117.0,Бензин,180000.0,2010.0,ASX,...,1643823556,2011.0,"{'gear_type': 'FORWARD_CONTROL', 'engine_type'...",механическая,NaN,2 владельца,Оригинал,передний,Левый,910000.0


In [167]:
# delete duplicates
df_train.drop_duplicates(inplace=True)
df_train.shape

(30387, 21)

In [166]:
sum(df_test.duplicated())

0

In [170]:
print_framed('TRAIN SET')
df_train.info()

print_framed('TEST SET')
df_test.info()

****************************************************************************************************
****************************************     TRAIN SET     ****************************************
****************************************************************************************************
<class 'pandas.core.frame.DataFrame'>
Int64Index: 30387 entries, 0 to 42304
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   bodyType             30387 non-null  object 
 1   brand                30387 non-null  object 
 2   color                30387 non-null  object 
 3   description          30387 non-null  object 
 4   engineDisplacement   30295 non-null  float64
 5   enginePower          30295 non-null  float64
 6   fuelType             30295 non-null  object 
 7   mileage              30387 non-null  float64
 8   modelDate            30387 non-null  float64
 9   model_name           30387 non-nul

Although the size of cleaned train is not so large it is although not so small, and for future models in this study project we want to use our own parsed train data.

## EDA and existing feature transformation

## New feature engineering

# Trials

In [192]:
%lsmagic

Available line magics:
%alias  %alias_magic  %autoawait  %autocall  %automagic  %autosave  %bookmark  %cache  %cd  %clear  %cls  %colors  %conda  %config  %connect_info  %copy  %ddir  %debug  %dhist  %dirs  %doctest_mode  %echo  %ed  %edit  %env  %gui  %hist  %history  %killbgscripts  %ldir  %less  %load  %load_ext  %loadpy  %logoff  %logon  %logstart  %logstate  %logstop  %ls  %lsmagic  %macro  %magic  %matplotlib  %mkdir  %more  %notebook  %page  %pastebin  %pdb  %pdef  %pdoc  %pfile  %pinfo  %pinfo2  %pip  %popd  %pprint  %precision  %prun  %psearch  %psource  %pushd  %pwd  %pycat  %pylab  %qtconsole  %quickref  %recall  %rehashx  %reload_ext  %ren  %rep  %rerun  %reset  %reset_selective  %rmdir  %run  %save  %sc  %set_env  %store  %sx  %system  %tb  %time  %timeit  %unalias  %unload_ext  %who  %who_ls  %whos  %xdel  %xmode

Available cell magics:
%%!  %%HTML  %%SVG  %%bash  %%capture  %%cmd  %%debug  %%file  %%html  %%javascript  %%js  %%latex  %%markdown  %%perl  %%prun  %%pypy  %

In [193]:
# %%timeit -r 10 -n 10
# time.sleep(0.1)


In [193]:
%%script NotExecute --no-raise-error

class StopExecution(Exception):
    def _render_traceback_(self):
        pass

raise StopExecution